In [1]:
import numpy as np 
from scipy.special import expit as sigmoid

In [2]:
class LSTM_demo():
    # Input size: 9  Hidden size: 2  Num layers: 1
    def __init__(self):
        # Forget gate weights
        self.W_hf = np.array([[0, 0], [0, 0]])
        self.B_hf = np.array([50, 50])
        self.W_xf = np.array([[0, 0, 0, 0, 0, 0, 0, -1000, -1000], [0, 0, 0, 0, 0, 0, 0, -1000, 0]])
        self.B_xf = np.array([50, 50])
        # Input gate weights
        self.W_hi = np.array([[0, 0], [0, 0]])
        self.B_hi = np.array([0, 0])
        self.W_xi = np.array([[50, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 50]])
        self.B_xi = np.array([0, 0])
        self.W_hl = np.array([[0, 0], [0, 0]])
        self.B_hl = np.array([0, 0])
        self.W_xl = np.array([[50, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 50]])
        self.B_xl = np.array([0, 0])
        # Output gate weights
        self.W_ho = np.array([[0, -500], [0, 0]])
        self.B_ho = np.array([0, 0])
        self.W_xo = np.array([[-1.33, 50, -0.78, -0.34, 0.5, 1.33, 0.01, 0, 0], [0, 0.65, -0.78, -0.34, 0.5, 0.08, 0, 0, -1.33]])
        self.B_xo = np.array([0, 0])
    
    def forget_gate(self, x, h, Weights_hf, Bias_hf, Weights_xf, Bias_xf, prev_cell_state): pass
    
    def input_gate(self, x, h, Weights_hi, Bias_hi, Weights_xi, Bias_xi, Weights_hl, Bias_hl, Weights_xl, Bias_xl): pass
    
    def cell_state(self, forget_gate_output, input_gate_output): pass
    
    def output_gate(self, x, h, Weights_ho, Bias_ho, Weights_xo, Bias_xo, cell_state): pass
    
    def classifier(self, h):
        if np.around(h[0], 3) == 0.208:
            return [0, 0, 0, 0, 0, 0, 1, 0, 0]
        elif np.around(h[1], 3) == 0.208:
            return [0, 0, 0, 0, 0, 0, 0, 1, 0]
        elif np.around(h[0]+h[1], 2) == 0.76:
            return [1, 0, 0, 0, 0, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.16:
            return [0, 1, 0, 0, 0, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.20:
            return [0, 0, 1, 0, 0, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.30:
            return [0, 0, 0, 1, 0, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.40:
            return [0, 0, 0, 0, 1, 0, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.60:
            return [0, 0, 0, 0, 0, 1, 0, 0, 0]
        elif np.around(h[0]+h[1], 2) == 0.50:
            return [0, 0, 0, 0, 0, 0, 0, 0, 1]
        else:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0]
        
    def __call__(self, x, h_c):
        h = h_c[0]
        c = h_c[1]
        
        f = self.forget_gate(x, h, self.W_hf, self.B_hf, self.W_xf, self.B_xf, c)
        i = self.input_gate(x, h, self.W_hi, self.B_hi, self.W_xi, self.B_xi, \
                             self.W_hl, self.B_hl, self.W_xl, self.B_xl,)
        c = self.cell_state(f, i)
        h = self.output_gate(x, h, self.W_ho, self.B_ho, self.W_xo, self.B_xo, c)
        
        h_c = (h, c)
        output = self.classifier(h)
        
        return output, h_c

In [3]:
class LSTM(LSTM_demo):
    def __init__(self):
        super().__init__()
        
    def forget_gate(self, x, h, Weights_hf, Bias_hf, Weights_xf, Bias_xf, prev_cell_state):
        hidden = np.dot(Weights_hf, h)+Bias_hf
        new = np.dot(Weights_xf, x)+Bias_xf
        total = hidden+new
        total_sig = sigmoid(total)
        result = total_sig*prev_cell_state
        return result
    
    def input_gate(self, x, h, Weights_hi, Bias_hi, Weights_xi, Bias_xi, Weights_hl, Bias_hl, Weights_xl, Bias_xl):
        hidden_i = np.dot(Weights_hi, h)+Bias_hi
        new_i = np.dot(Weights_xi, x)+Bias_xi
        total_i = hidden_i+new_i
        total_i_sig = sigmoid(total_i)
        
        hidden_l = np.dot(Weights_hl, h)+Bias_hl
        new_l = np.dot(Weights_xl, x)+Bias_xl
        total_l = hidden_l+new_l
        total_l_tanh = np.tanh(total_l)
        
        result = total_i_sig*total_l_tanh
        return result
    
    def cell_state(self, forget_gate_output, input_gate_output):
        result = forget_gate_output+input_gate_output
        ##result = np.row_stack((forget_gate_output, input_gate_output))
        return result
    
    def output_gate(self, x, h, Weights_ho, Bias_ho, Weights_xo, Bias_xo, cell_state):
        cell = np.tanh(cell_state)
        
        hidden_o = np.dot(Weights_ho, h)+Bias_ho
        new_o = np.dot(Weights_xo, x)+Bias_xo
        total_o = hidden_o+new_o
        total_o_sig = sigmoid(total_o)
        result = total_o_sig*cell
        return result
        
        

In [4]:
def word2vector(word):
    if word == '老':
        vector = [1, 0, 0, 0, 0, 0, 0, 0, 0]
    elif word == '吾':
        vector = [0, 1, 0, 0, 0, 0, 0, 0, 0]
    elif word == '以':
        vector = [0, 0, 1, 0, 0, 0, 0, 0, 0]
    elif word == '及':
        vector = [0, 0, 0, 1, 0, 0, 0, 0, 0]
    elif word == '人':
        vector = [0, 0, 0, 0, 1, 0, 0, 0, 0]
    elif word == '之':
        vector = [0, 0, 0, 0, 0, 1, 0, 0, 0]
    elif word == '，':
        vector = [0, 0, 0, 0, 0, 0, 1, 0, 0]
    elif word == '。':
        vector = [0, 0, 0, 0, 0, 0, 0, 1, 0]
    elif word == '幼':
        vector = [0, 0, 0, 0, 0, 0, 0, 0, 1]
    else:
        vector = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    return vector

In [5]:
def vector2word(vector):
    if vector == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
        word = '老'
    elif vector == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
        word = '吾'
    elif vector == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
        word = '以'
    elif vector == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
        word = '及'
    elif vector == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
        word = '人'
    elif vector == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
        word = '之'
    elif vector == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
        word = '，'
    elif vector == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
        word = '。'
    elif vector == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
        word = '幼'
    else:
        word = 'UNKNOWN WORD'
    return word

In [6]:
lstm = LSTM()
h_c = (np.zeros(2), np.zeros(2))

In [7]:
string = ""
x = '老'
string = string + x

while(1):
    print(string)
    x = word2vector(x)
    output, h_c = lstm(x, h_c)
    x = vector2word(output)
    if x == 'UNKNOWN WORD': break
    string = string + x

老
老吾
老吾老
老吾老以
老吾老以及
老吾老以及人
老吾老以及人之
老吾老以及人之老
老吾老以及人之老，
老吾老以及人之老，幼
老吾老以及人之老，幼吾
老吾老以及人之老，幼吾幼
老吾老以及人之老，幼吾幼以
老吾老以及人之老，幼吾幼以及
老吾老以及人之老，幼吾幼以及人
老吾老以及人之老，幼吾幼以及人之
老吾老以及人之老，幼吾幼以及人之幼
老吾老以及人之老，幼吾幼以及人之幼。
